# RDD练习

In [1]:
import findspark
import pyspark 
from pyspark import SparkContext, SparkConf
#指定spark_home为刚才的解压路径,指定python路径
spark_home = "/home/parallels/Desktop/work/spark-3.3.2-bin-hadoop3"
python_path = "/home/parallels/anaconda3/bin/python3"
findspark.init(spark_home,python_path)

conf = SparkConf().setAppName("test").setMaster("local[*]")
sc = SparkContext(conf=conf)
print(pyspark.__version__)

23/05/18 08:33:23 WARN Utils: Your hostname, ubuntu-linux-22-04-desktop resolves to a loopback address: 127.0.1.1; using 10.211.55.3 instead (on interface enp0s5)
23/05/18 08:33:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/18 08:33:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
3.3.2


## 1、求平均数
任务：求data的平均值
data = [1,5,7,10,23,20,6,5,10,7,10]

### 我的解法

In [5]:
rdd = sc.parallelize([1,5,7,10,23,20,6,5,10,7,10])
sum = sc.accumulator(0)
count = sc.accumulator(0)

def get_average(x):
    sum.add(x)
    count.add(1)

rdd.foreach(get_average)
print(sum.value/count.value)

9.454545454545455


### 答案
怎么没我的快？🤕

In [6]:
#任务：求data的平均值
data = [1,5,7,10,23,20,6,5,10,7,10]

rdd_data = sc.parallelize(data)
s = rdd_data.reduce(lambda x,y:x+y+0.0)
n = rdd_data.count()
avg = s/n
print("average:",avg)

average: 9.454545454545455


## 2、求众数
任务：求data中出现次数最多的数
data =  [1,5,7,10,23,20,7,5,10,7,10]

### 我的解法

In [16]:
from operator import add
rdd = sc.parallelize([1,5,7,10,23,20,7,5,10,7,10])
max_rdd = rdd.map(lambda x: (x,1)).reduceByKey(add).sortBy(lambda x: x[1],False)
max_num = max_rdd.first()[1]
result = max_rdd.filter(lambda x: x[1] == max_num).map(lambda x: x[0]).collect()
count = 0
sum = 0
for i in result:
    count += 1
    sum += i
print("出现次数最多的数字是：",sum/count)
    


出现次数最多的数字是： 8.5


### 答案

In [7]:
data =  [1,5,7,10,23,20,7,5,10,7,10]

rdd_data = sc.parallelize(data)
rdd_count = rdd_data.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
max_count = rdd_count.map(lambda x:x[1]).reduce(lambda x,y: x if x>=y else y)
rdd_mode = rdd_count.filter(lambda x:x[1]==max_count).map(lambda x:x[0])
mode = rdd_mode.reduce(lambda x,y:x+y+0.0)/rdd_mode.count()
print("mode:",mode)

mode: 8.5


## 3、求TopN
任务：有一批学生信息表格，包括name,age,score, 找出score排名前3的学生, score相同可以任取
students = [("LiLei",18,87),("HanMeiMei",16,77),("DaChui",16,66),("Jim",18,77),("RuHua",18,50)]
n = 3

### 我的解法

In [19]:
students = [("LiLei",18,87),("HanMeiMei",16,77),("DaChui",16,66),("Jim",18,77),("RuHua",18,50)]
n = 3

def getTopN(students,n):
    rdd = sc.parallelize(students)
    sort_rdd = rdd.sortBy(lambda x:x[2],False)
    topN = sort_rdd.take(n)
    return topN

print(getTopN(students,n))

[('LiLei', 18, 87), ('HanMeiMei', 16, 77), ('Jim', 18, 77)]


### 答案

In [20]:
students = [("LiLei",18,87),("HanMeiMei",16,77),("DaChui",16,66),("Jim",18,77),("RuHua",18,50)]
n = 3

rdd_students = sc.parallelize(students)
rdd_sorted = rdd_students.sortBy(lambda x:x[2],ascending = False)

students_topn = rdd_sorted.take(n)
print(students_topn)


[('LiLei', 18, 87), ('HanMeiMei', 16, 77), ('Jim', 18, 77)]


## 4、排序并返回序号
任务：排序并返回序号, 大小相同的序号可以不同
data = [1,7,8,5,3,18,34,9,0,12,8]

### 我的解法

In [23]:
data = [1,7,8,5,3,18,34,9,0,12,8]

rdd = sc.parallelize(data)
rdd.sortBy(lambda x:x).zipWithIndex().map(lambda x: (x[1],x[0])).collect()

[(0, 0),
 (1, 1),
 (2, 3),
 (3, 5),
 (4, 7),
 (5, 8),
 (6, 8),
 (7, 9),
 (8, 12),
 (9, 18),
 (10, 34)]

### 答案

In [ ]:
#任务：按从小到大排序并返回序号, 大小相同的序号可以不同
data = [1,7,8,5,3,18,34,9,0,12,8]

rdd_data = sc.parallelize(data)
rdd_sorted = rdd_data.map(lambda x:(x,1)).sortByKey().map(lambda x:x[0])
rdd_sorted_index = rdd_sorted.zipWithIndex()

print(rdd_sorted_index.collect())

## 5、二次排序
任务：有一批学生信息表格，包括name,age,score
首先根据学生的score从大到小排序，如果score相同，根据age从大到小
students = [("LiLei",18,87),("HanMeiMei",16,77),("DaChui",16,66),("Jim",18,77),("RuHua",18,50)]

### 我的解法（❌）

In [ ]:
students = [("LiLei",18,87),("HanMeiMei",16,77),("DaChui",16,66),("Jim",18,77),("RuHua",18,50)]
rdd = sc.parallelize(students)
rdd.sortBy(lambda x:x[2],False).sortBy(lambda x:[x[1]]).collect()

### 答案

In [25]:
#任务：有一批学生信息表格，包括name,age,score
#首先根据学生的score从大到小排序，如果score相同，根据age从大到小

students = [("LiLei",18,87),("HanMeiMei",16,77),("DaChui",16,66),("Jim",18,77),("RuHua",18,50)]
rdd_students = sc.parallelize(students)

In [26]:
%%writefile student.py
#为了在RDD中使用自定义类，需要将类的创建代码其写入到一个文件中，否则会有序列化错误
class Student:
    def __init__(self,name,age,score):
        self.name = name
        self.age = age
        self.score = score
    def __gt__(self,other):
        if self.score > other.score:
            return True
        elif self.score==other.score and self.age>other.age:
            return True
        else:
            return False

Writing student.py


In [27]:
from student import Student

rdd_sorted = rdd_students \
    .map(lambda t:Student(t[0],t[1],t[2]))\
    .sortBy(lambda x:x,ascending = False)\
    .map(lambda student:(student.name,student.age,student.score))

#参考方案：此处巧妙地对score和age进行编码来表达其排序优先级关系，除非age超过100000，以下逻辑无错误。
#rdd_sorted = rdd_students.sortBy(lambda x:100000*x[2]+x[1],ascending=False)

rdd_sorted.collect()

[('LiLei', 18, 87),
 ('Jim', 18, 77),
 ('HanMeiMei', 16, 77),
 ('DaChui', 16, 66),
 ('RuHua', 18, 50)]

## 6、连接操作
任务：已知班级信息表和成绩表，找出班级平均分在75分以上的班级
班级信息表包括class,name,成绩表包括name,score

classes = [("class1","LiLei"), ("class1","HanMeiMei"),("class2","DaChui"),("class2","RuHua")]
scores = [("LiLei",76),("HanMeiMei",80),("DaChui",70),("RuHua",60)]

### 我的解法

In [31]:
classes = [("class1","LiLei"), ("class1","HanMeiMei"),("class2","DaChui"),("class2","RuHua")]
scores = [("LiLei",76),("HanMeiMei",80),("DaChui",70),("RuHua",60)]
rdd_classes = sc.parallelize(classes).map(lambda x:(x[1],x[0]))
rdd_scores = sc.parallelize(scores)
rdd_join = rdd_classes.join(rdd_scores).groupBy(lambda x:x[1][0])

[('class2', <pyspark.resultiterable.ResultIterable at 0xffff683e0c10>),
 ('class1', <pyspark.resultiterable.ResultIterable at 0xffff683fad40>)]

### 答案

In [32]:
#任务：已知班级信息表和成绩表，找出班级平均分在75分以上的班级
#班级信息表包括class,name,成绩表包括name,score

classes = [("class1","LiLei"), ("class1","HanMeiMei"),("class2","DaChui"),("class2","RuHua")]
scores = [("LiLei",76),("HanMeiMei",80),("DaChui",70),("RuHua",60)]

rdd_classes = sc.parallelize(classes).map(lambda x:(x[1],x[0]))
rdd_scores = sc.parallelize(scores)
rdd_join = rdd_scores.join(rdd_classes).map(lambda t:(t[1][1],t[1][0]))

def average(iterator):
    data = list(iterator)
    s = 0.0
    for x in data:
        s = s + x
    return s/len(data)

rdd_result = rdd_join.groupByKey().map(lambda t:(t[0],average(t[1]))).filter(lambda t:t[1]>75)
print(rdd_result.collect())

[('class1', 78.0)]


## 7、分组求众数
任务：有一批学生信息表格，包括class和age。求每个班级学生年龄的众数。
students = [("class1",15),("class1",15),("class2",16),("class2",16),("class1",17),("class2",19)]

### 我的解法

In [9]:
from operator import add
students = [("class1",15),("class1",15),("class2",16),("class2",16),("class1",17),("class2",19)]
rdd = sc.parallelize(students)
rdd_group = rdd.groupByKey().map(lambda x: (x,1)).reduceByKey(add)
# .map(lambda t:(t[0],list(t[1])))

def getMostNum(iterator):
    data = list(iterator)
    # 求众数
    

    # data.sort(reverse=True)
    # count = 0
    # most_num = data[0]
    # for x in data:
    #     if x == most_num:
    #         count = count + 1
    #     else:
    #         count = 1
    #         most_num = x
    # return most_num
rdd_group.map(lambda t:(t[0],getMostNum(t[1]))).collect()


[('class1', 15), ('class2', 16)]

### 答案

In [10]:
students = [("class1",15),("class1",15),("class2",16),("class2",16),("class1",17),("class2",19)]
def mode(arr):
    dict_cnt = {}
    for x in arr:
        dict_cnt[x] = dict_cnt.get(x,0)+1
    max_cnt = max(dict_cnt.values())
    most_values = [k for k,v in dict_cnt.items() if v==max_cnt]
    s = 0.0
    for x in most_values:
        s = s + x
    return s/len(most_values)

rdd_students = sc.parallelize(students)
rdd_classes = rdd_students.aggregateByKey([],lambda arr,x:arr+[x],lambda arr1,arr2:arr1+arr2)
rdd_mode = rdd_classes.map(lambda t:(t[0],mode(t[1])))

print(rdd_mode.collect())

[('class1', 15.0), ('class2', 16.0)]
